# Classifictation model 

In [15]:
%reset -f

In [16]:
# Libraries

import os
from PIL import Image
import itertools

import cv2
from skimage import io, color, filters
import numpy as np 
import pandas as pd 
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import warnings
warnings.filterwarnings('ignore')

print('Modules Successfully Loaded')

Modules Successfully Loaded


## Dataset 

In [17]:
data_dir= "./Multi-class Weather Dataset"
paths = []
labels = []
folds = os.listdir(data_dir)
for fold in folds :
    condition_path = data_dir +"/" + fold
    all_pic = os.listdir(condition_path)
    for each_pic in all_pic:
        each_pic_path = condition_path +"/" + each_pic
        paths.append(each_pic_path)
        labels.append(fold)
        
pseries = pd.Series(paths, name='Picture Path')
lseries = pd.Series(labels, name='Label')

df = pd.concat([pseries, lseries], axis =1)
df.head()

,Picture Path,Label
0,./Multi-class Weather Dataset/Cloudy/cloudy1.jpg,Cloudy
1,./Multi-class Weather Dataset/Cloudy/cloudy10.jpg,Cloudy
2,./Multi-class Weather Dataset/Cloudy/cloudy100...,Cloudy
3,./Multi-class Weather Dataset/Cloudy/cloudy101...,Cloudy
4,./Multi-class Weather Dataset/Cloudy/cloudy102...,Cloudy


In [18]:
# Convert the labels to French
df['Label'] = df['Label'].replace({'Cloudy': 'Nuageux', 'Rain': 'Pluvieux', 'Shine': 'Ensoleillé', 'Sunrise': 'Lever du soleil'})
df

,Picture Path,Label
0,./Multi-class Weather Dataset/Cloudy/cloudy1.jpg,Nuageux
1,./Multi-class Weather Dataset/Cloudy/cloudy10.jpg,Nuageux
2,./Multi-class Weather Dataset/Cloudy/cloudy100...,Nuageux
3,./Multi-class Weather Dataset/Cloudy/cloudy101...,Nuageux
4,./Multi-class Weather Dataset/Cloudy/cloudy102...,Nuageux
...,...,...
1120,./Multi-class Weather Dataset/Sunrise/sunrise9...,Lever du soleil
1121,./Multi-class Weather Dataset/Sunrise/sunrise9...,Lever du soleil
1122,./Multi-class Weather Dataset/Sunrise/sunrise9...,Lever du soleil
1123,./Multi-class Weather Dataset/Sunrise/sunrise9...,Lever du soleil


In [19]:

# Function to get image dimensions
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
    return width, height 

# Add columns to dataframe
df['width'], df['height'] = zip(*df['Picture Path'].map(get_image_dimensions))

df.head()

,Picture Path,Label,width,height
0,./Multi-class Weather Dataset/Cloudy/cloudy1.jpg,Nuageux,600,400
1,./Multi-class Weather Dataset/Cloudy/cloudy10.jpg,Nuageux,271,186
2,./Multi-class Weather Dataset/Cloudy/cloudy100...,Nuageux,259,194
3,./Multi-class Weather Dataset/Cloudy/cloudy101...,Nuageux,338,149
4,./Multi-class Weather Dataset/Cloudy/cloudy102...,Nuageux,800,377


# Train Test Split (80% - 20%)

In [20]:
train_set, test_set = train_test_split(df, test_size=0.2, shuffle=True, stratify=df["Label"], random_state=42)

In [21]:
train_set.head()

,Picture Path,Label,width,height
646,./Multi-class Weather Dataset/Shine/shine217.jpg,Ensoleillé,270,169
892,./Multi-class Weather Dataset/Sunrise/sunrise2...,Lever du soleil,3200,1200
157,./Multi-class Weather Dataset/Cloudy/cloudy240...,Nuageux,300,138
682,./Multi-class Weather Dataset/Shine/shine25.jpg,Ensoleillé,852,480
224,./Multi-class Weather Dataset/Cloudy/cloudy300...,Nuageux,300,149


In [22]:
test_set.head()

,Picture Path,Label,width,height
412,./Multi-class Weather Dataset/Rain/rain20.jpg,Pluvieux,620,413
645,./Multi-class Weather Dataset/Shine/shine216.jpg,Ensoleillé,230,153
90,./Multi-class Weather Dataset/Cloudy/cloudy180...,Nuageux,261,175
717,./Multi-class Weather Dataset/Shine/shine53.jpg,Ensoleillé,284,177
393,./Multi-class Weather Dataset/Rain/rain183.jpg,Pluvieux,630,446


# Image Data generator

In [23]:
batch_size = 64
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

print("Train Set:")
train_gen = tr_gen.flow_from_dataframe(train_set, x_col='Picture Path', y_col='Label', target_size=img_size, class_mode='categorical',
                                       color_mode='rgb', shuffle=True, batch_size=batch_size)


print("Test Set:")
test_gen = ts_gen.flow_from_dataframe(test_set, x_col='Picture Path', y_col='Label', target_size=img_size, class_mode='categorical',
                                      color_mode='rgb', shuffle=False, batch_size=batch_size)


Train Set:
Found 900 validated image filenames belonging to 4 classes.
Test Set:
Found 225 validated image filenames belonging to 4 classes.


# Caracteristiques

In [28]:
def resize_and_crop_top(path, new_width=507, new_height=335):
    """
    Resize the image to the specified dimensions and keep only the top 70% of the image.
    
    Args:
    - image: numpy array representing the image.
    - new_width: desired width after resizing.
    - new_height: desired height after resizing.
    
    Returns:
    - Resized and cropped image.
    """
    image = io.imread(path)
    # Resize the image to the desired dimensions without interpolation
    resized_image = cv2.resize(image, (new_width, new_height))
    
    # Calculate the new height to keep only the top 70%
    cropped_height = int(new_height * 0.7)
    
    # Crop the image to keep only the top 70%
    cropped_image = resized_image[:cropped_height, :]
    
    return cropped_image

In [29]:

def resize_images(folder_path, target_size):

    # Parcourir toutes les images dans le dossier
    for filename in os.listdir(folder_path):
        if filename is not None:
            image_path = os.path.join(folder_path, filename)
            
            # Vérifier si le fichier est une image
            if os.path.isfile(image_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp',)) :
                
                # Charger l'image
                image = cv2.imread(image_path)

                # Vérifier si l'image est correctement chargée
                if image is not None:
                    # Redimensionner l'image à la nouvelle taille
                    resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)
                    
                    # Enregistrer l'image redimensionnée
                    cv2.imwrite(image_path, resized_image)
                else:
                    print(f"Impossible de charger l'image : {image_path}")

    print("Toutes les images ont été redimensionnées avec succès.")

In [27]:
def get_luminance(image):
    # Vérifier si l'image a une quatrième couche d'alpha
    if image.shape[2] == 4:
        # Supprimer la quatrième couche d'alpha
        image = image[:, :, :3]
    gray_image = color.rgb2gray(image)
    luminance_mean = np.std(gray_image)
    return luminance_mean  

    
def get_histogram(image):
    
    red_channel = image[:, :, 0]
    green_channel = image[:, :, 1]
    blue_channel = image[:, :, 2]
    
    # Calcul des histogrammes marginaux
    plt.figure(figsize=(8, 5))
    
    red_hist, red_bins, _ = plt.hist(red_channel.ravel(), bins=256, color='red', alpha=0.5, label='Rouge')
    green_hist, green_bins, _ = plt.hist(green_channel.ravel(), bins=256, color='green', alpha=0.5, label='Vert')
    blue_hist, blue_bins, _ = plt.hist(blue_channel.ravel(), bins=256, color='blue', alpha=0.5, label='Bleu')
    plt.title('Histogramme marginal des couleurs RGB')
    plt.xlabel('Intensité de couleur')
    plt.ylabel('Fréquence')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
  # Calcul des moyennes de chaque canal
    
    red_mean = np.average(red_bins, weights=red_hist)
    green_mean = np.average(green_bins, weights=green_hist)
    blue_mean = np.average(blue_bins, weights=blue_hist)

    
    return [red_mean, green_mean, blue_mean]


In [30]:
# preprocess + process

def calculate_luminance(path):
    img = resize_and_crop_top(path)
    try:
        return get_luminance(img)
    except:
        return None

def calculate_histogram(path):
    img = resize_and_crop_top(path)
    try:
        return get_histogram(img)
    except:
        return None